In [3]:
!kaggle datasets download -d akash2907/bird-species-classification

100%|██████████████████████████████████████▉| 1.37G/1.37G [00:13<00:00, 200MB/s]
100%|███████████████████████████████████████| 1.37G/1.37G [00:13<00:00, 112MB/s]


In [5]:
!mkdir -p data
!mv bird-species-classification.zip data
!yes | unzip data/bird-species-classification.zip -d data

Archive:  data/bird-species-classification.zip
replace data/test_data/test_data/blasti/DSC_6396.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: data/test_data/test_data/blasti/DSC_6396.jpg  
replace data/test_data/test_data/blasti/DSC_6397.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: data/test_data/test_data/blasti/DSC_6397.jpg  
replace data/test_data/test_data/blasti/DSC_6398.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: data/test_data/test_data/blasti/DSC_6398.jpg  
replace data/test_data/test_data/blasti/DSC_6399.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: data/test_data/test_data/blasti/DSC_6399.jpg  
replace data/test_data/test_data/blasti/DSC_6400.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: data/test_data/test_data/blasti/DSC_6400.jpg  
replace data/test_data/test_data/blasti/DSC_6401.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: data/test_data/test_data/blasti/DSC_6401.jpg  
replace data/test_data/test_data/blasti/D

In [6]:
# Build a dataframe with the filepaths and labels for training data in train_data/train_data
import os
import pandas as pd

filepaths = []
labels = []
data_dir = 'data/train_data/train_data'
for root, dirs, files in os.walk(data_dir):
    for file in files:
        filepaths.append(os.path.join(root, file))
        labels.append(root.split('/')[-1])

filepaths = pd.Series(filepaths, name='Filepath')
labels = pd.Series(labels, name='Label')

df = pd.concat([filepaths, labels], axis=1)
df = pd.get_dummies(df, columns=['Label'])
df


,Filepath,Label_blasti,Label_bonegl,Label_brhkyt,Label_cbrtsh,Label_cmnmyn,Label_gretit,Label_hilpig,Label_himbul,Label_himgri,Label_hsparo,Label_indvul,Label_jglowl,Label_lbicrw,Label_mgprob,Label_rebimg,Label_wcrsrt
0,data/train_data/train_data/himgri/12152037683_...,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False
1,data/train_data/train_data/himgri/12266086526_...,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False
2,data/train_data/train_data/himgri/12029822653_...,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False
3,data/train_data/train_data/himgri/12265055185_...,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False
4,data/train_data/train_data/himgri/12265561764_...,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,data/train_data/train_data/cmnmyn/6154956165_6...,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
146,data/train_data/train_data/cmnmyn/5866682091_8...,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
147,data/train_data/train_data/cmnmyn/DSCN5784.jpg,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
148,data/train_data/train_data/cmnmyn/DSCN5790.jpg,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False


In [7]:
# Split the data into training and validation sets
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# Create a dataloader
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

class BirdDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.df.iloc[idx, 0]
        image = Image.open(img_path)
        label = self.df.iloc[idx, 1]
        label = label.astype(int)

        if self.transform:
            image = self.transform(image)

        return image, label

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = BirdDataset(train_df, transform=transform)
val_dataset = BirdDataset(val_df, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [10]:
# Create a resnet model
import torch.nn as nn
from torchvision import models

model = models.mobilenet_v2(pretrained=True)
# If you want to fine-tune only the top layer of the model, set as below
for param in model.parameters():
    param.requires_grad = False
num_ftrs = model.classifier[1].in_features
model.classifier = nn.Linear(num_ftrs, 250)

# Train the model
import torch.optim as optim
import torch.nn.functional as F

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

n_epochs = 10

for epoch in range(n_epochs):
    train_loss = 0.0
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    model.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * inputs.size(0)

    model.eval()
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        val_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs.data, 1)
        val_total += labels.size(0)
        val_correct += (predicted == labels).sum().item()

    train_loss = train_loss / len(train_loader.dataset)
    val_loss = val_loss / len(val_loader.dataset)
    val_accuracy = val_correct / val_total * 100

    print(f'Epoch: {epoch+1}/{n_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')



/home/vscode/.local/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/vscode/.local/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch: 1/10, Train Loss: 3.8387, Val Loss: 1.4272, Val Accuracy: 90.00%
Epoch: 2/10, Train Loss: 0.7204, Val Loss: 0.2427, Val Accuracy: 93.33%
Epoch: 3/10, Train Loss: 0.3098, Val Loss: 0.1536, Val Accuracy: 93.33%
Epoch: 4/10, Train Loss: 0.2577, Val Loss: 0.1067, Val Accuracy: 93.33%
Epoch: 5/10, Train Loss: 0.2004, Val Loss: 0.0819, Val Accuracy: 100.00%
Epoch: 6/10, Train Loss: 0.1351, Val Loss: 0.0893, Val Accuracy: 100.00%
Epoch: 7/10, Train Loss: 0.1164, Val Loss: 0.0654, Val Accuracy: 100.00%
Epoch: 8/10, Train Loss: 0.0953, Val Loss: 0.0414, Val Accuracy: 100.00%
Epoch: 9/10, Train Loss: 0.0851, Val Loss: 0.0274, Val Accuracy: 100.00%
Epoch: 10/10, Train Loss: 0.0783, Val Loss: 0.0211, Val Accuracy: 100.00%


In [11]:
# Savce the model to onnx format
torch.onnx.export(model, torch.randn(1, 3, 224, 224).to(device), 'bird_model.onnx', export_params=True, opset_version=10, do_constant_folding=True, input_names=['input'], output_names=['output'], dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}})
!mv bird_model.onnx data/bird_model.onnx

OnnxExporterError: Module onnx is not installed!